In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [32]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import os
import time

In [25]:
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421'

In [187]:
chrome_options = Options()  
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
page = driver.page_source
driver.close()
driver.quit()

In [188]:
html = BeautifulSoup(page, 'html.parser')

In [189]:
z = html.findAll('div', {'data-sephid':'5'})

In [201]:
page = requests.get('https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421')
html = BeautifulSoup(page.text, 'html.parser')
stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
stars

'4 stars'

In [199]:
review_score = html.find("div", {'data-comp':"ReviewsStats Box "})
review_score.span.get_text()

AttributeError: 'NoneType' object has no attribute 'span'

In [185]:
page = requests.get('https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421')
html = BeautifulSoup(page.text, 'html.parser')
skin_type = html.find('div', {'data-at':'product_tabs_section'})
ingredients = skin_type.findAll('div', {'aria-labelledby':True})[2]

splitted = ingredients.get_text(strip=False, separator=' |$%&*$| ')

splitted.split(' |$%&*$| ')[3].split(', ')

['Water',
 'Saccharomyces/Camellia Sinensis Leaf/Cladosiphon Okamuranus/Rice Ferment Filtrate*',
 'Dimethicone',
 'Propanediol',
 'Glycerin',
 'Diglycerin',
 'Diphenylsiloxy Phenyl Trimethicone',
 'Gold',
 'Belamcanda Chinensis Root Extract',
 'Rosa Multiflora Fruit Extract',
 'Houttuynia Cordata Extract',
 'Sophora Angustifolia Root Extract',
 'Sodium Hyaluronate',
 'Lecithin',
 'Pistacia Lentiscus (Mastic) Gum',
 'Sodium Chloride',
 'Sodium Citrate',
 'Mica',
 'Peg-9 Polydimethylsiloxyethyl Dimethicone',
 'Dimethicone/Peg-10/15 Crosspolymer',
 'Dimethicone/Phenyl Vinyl Dimethicone Crosspolymer',
 'Disodium Edta',
 'Tin Oxide',
 'Titanium Dioxide',
 'Butylene Glycol',
 'Ethylhexylglycerin',
 'Parfum/Fragrance',
 'Alcohol',
 'Phenoxyethanol. *Hadasei-3.\r\n']

In [208]:
# Parse for item features
def get_static_meta(item_url:str) -> dict:
    '''Returns item/product features from static section of given sephora url'''
    # GET Request to URL
    page = requests.get(item_url)
    # Create BS4 Object
    html = BeautifulSoup(page.text, "html.parser")
    
    # Find Name and Brand
    brand, name = (x.get_text() for x in html.find("h1", attrs={"data-comp":True}))
    
    # Find n_loves (product likes)
    n_loves = html.find('span', attrs={"data-at":"product_love_count"}).get_text()
    
    stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
    
    return brand, name, n_loves, stars

In [97]:
def get_dynamic_meta(url:str) -> tuple:
    '''Returns item/product features from scoll-loaded section of given sephora url'''
    # Init Chrome Driver Options
    chrome_options = Options()  
    # Run headless arg to avoid GUI
    chrome_options.add_argument("--headless")
    # Init Driver Instance -> load options
    driver = webdriver.Chrome(options=chrome_options)
    
    # Call url via driver
    driver.get(url)
    # Scroll down to load REVEIW section
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    page = driver.page_source
    driver.close()
    driver.quit()
    
    

In [98]:
# brand, name, n_loves = get_static_meta('https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421')

# brand, name, n_loves

In [99]:
get_dynamic_meta(url)

In [210]:
def sephora_scrape(url:str) -> dict:
    '''Returns dictionary of needed features parsed from sephora.com link using selenium for dynamic page loading'''
    # Statically scrape product meta info
    brand, name, n_loves, stars = get_static_meta(url) 
    
    # Dynamically scrape product review section
    

In [211]:
x, y, z, sephora_scrape('https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218')